In [ ]:
%matplotlib

In [ ]:
import mne
from pprint import pprint
import json
import csv
import numpy as np
import pandas as pd
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
from pathlib import Path
import re

# Part one
## EEG preprocessing

### Step 1: Load data
### Step 2: ICA
### Step 3: Save ICAed data

# Part two
## EEG ERP (epochs to evoked)

### Step 4: Change trigger labels
### Step 5: Label epoch
### Step 6: Save epoch

# Part o
## EEG ERP (statistics)

### Step 7: Gather all subjects' epochs
### Step 8: Grand average
### Step 9: Draw ERP
### Step 10: Report ERP statistics

In [ ]:
# Step 1: Open the cnt file form of EEG data
# >> /Volumes/Neurolang_1/Master Program/New_Thesis_topic/Experiments_Results/EEG_ESLs

root_data_path = Path('/Volumes/Neurolang_1/Master Program/New_Thesis_topic/Experiments_Results')
eeg_data_path = root_data_path / "EEG_ESLs"

sub_idINT = 30

cntFile = eeg_data_path / Path('S%.3d_Alice.cnt' %sub_idINT)  #'S001_Alice.cnt'     # %.3d >> d = digits, 3 = 3digits

# S020_Alice.cnt

'''
S001_Alice.cnt: 最外圈的（編號尾數7的）沒有很毛，所以不exclude
'''

raw = mne.io.read_raw_cnt(cntFile, preload=True)

#raw.load_data()
#raw.plot()
print(raw.info)
print(raw.info["ch_names"])
print(raw.info["nchan"])



# Downsample to 100Hz, take off the filter, and exclude certain channels
n_raw = raw.copy()
n_raw.filter(0.1, 200, phase= 'zero-double')  # 再加0.1~30** / 1~30(訊號被扭曲的機率會更大) ，再重跑ICA  5-20Hz
n_raw.resample(sfreq=100)
n_raw.set_eeg_reference(['M1', 'M2'])
n_raw.pick_types(eeg=True, stim=False, exclude=['M1', 'M2', 'O1', 'OZ', 'O2', 'CB1', 'CB2', 'VEO', 'HEO', 'Audio'])
# remove dc-offset                                                     # 'VEO', 'HEO', 'Audio' >> new layouts  #'HEOG', 'VEOG'

# set montage????? yes!!!
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')  # for NCU-ICN's Room 608 EEG
n_raw.set_montage(ten_twenty_montage, match_case=False)
n_raw.plot()


In [ ]:
n_raw.plot_sensors() #(ch_type="eeg")

In [ ]:
## WE NEED TO USE THE CUT FILE TO PERFORM ICA!!!

# ICA
ica_ = ICA(n_components = 15, random_state=97)
ica_.fit(n_raw)

# plot ICA
#n_raw.plot_sensors(sphere = (0.0, 0.02, 0.0, 0.065))

# Exclude certain components
ica_.plot_sources(n_raw)
ica_.plot_components(sphere = (0.0, 0.02, 0.0, 0.065))  # What is this sphere for??
#ica_.plot_overlay(n_raw, exclude=[0, 8])

In [ ]:
for i in range(15):
    #ica_.plot_properties(n_raw, picks=[i])
    ica_.plot_overlay(n_raw, exclude=[i])

In [ ]:
#c_all_tmpLIST = list(range(0, 15))
#print(c_all_tmpLIST)

ica_.plot_overlay(n_raw, exclude=[0, 1, 10])

In [ ]:
#### What the hack is this?????

# This is the properties of each component
#ica_.plot_properties(n_raw, picks=[0, 1, 2, 3, 4, 5, 6, 7, 8])

# apply the cleaned ICA to the data 
raw_ICAed = n_raw.copy()
ica_.exclude = [0, 1, 3, 5, 7, 10]

ica_.plot_overlay(raw_ICAed, exclude=ica_.exclude)

ica_.apply(raw_ICAed)
#raw_ICAed.plot()

In [ ]:
# Save the ICAed data into fif file
#raw_ICAed.save(eeg_data_path/'Alice_ESL_ICAed_fif'/Path('_S%.3d_ICAed_raw.fif' %sub_idINT), overwrite=True)
raw_ICAed.save(eeg_data_path/'Alice_ESL_ICAed_fif'/Path('S%.3d_ICAed_raw.fif' %sub_idINT), overwrite=True)

# EEG ERP (epochs to evoked)

### Step 4: Change trigger labels
### Step 5: Label epoch
### Step 6: Save epoch

In [ ]:
# Creating the epochs(The Original one_without the tag of H&L-CD )
events = mne.find_events(n_LDT_raw, stim_channel='STI 014')  #just checking what stim_channel documenting what.
event_id = {'word_onset': 1}  

tmin = -0.1         								# pre stimulis interval (in seconds) #
tmax = 1       								# post stimulus interval #

baseline = (-0.1, 0)  # tmin ~ 0 , -0.1s-0s   
reject = dict(mag=4e-12)  # 5e-12 dropped so many

# Load in the channels layouts
kit_lay = mne.channels.read_layout("KIT-157")

#n_LDT_raw.load_data().pick_types(meg=True, stim=True).filter(0, 30, phase='zero-double').resample(500)
epoch_LDT = mne.Epochs(n_LDT_raw, events, event_id, tmin, tmax, baseline=baseline
                    , reject=reject,preload=True)

#epoch_LDT.plot()

In [ ]:
# get to know the reason of bad epoch drop
print(epoch_LDT.drop_log)

In [ ]:
# Load in the csv file of word onset
print(root_data_path)
csv_data = root_data_path / "All_PLDT_raw_results.csv"  # self-made LMM data form

sub_ansLIST = []
with open(csv_data, "r", encoding="UTF-8") as f:
    fileLIST = f.read().split("\n")
    fileLIST.pop(0)  # exclude the headlines
    fileLIST.pop(-1) # exclude the last one (blank)
    
    #pprint(fileLIST[0:10][2])
    pprint(fileLIST)
    print(len(fileLIST))

In [ ]:
# set up the sub_id into string
tmp_id = "%.3d" %sub_id   # 5678 =! 005~008 >> use %d
print(tmp_id)
print(type(tmp_id))

# create blank LIST for later use
sub_ansLIST = []

# create the LIST of the target sub's answer
for row in fileLIST:
    row = row.split(",")
    sub_id_STR = row[0]
    #print(sub_id_STR)
    #print(type(sub_id_STR))
    
    # check the sub_id in the whole dataset
    if sub_id_STR == tmp_id:
        sub_ansLIST.append(row)
    else:
        pass
        
pprint(sub_ansLIST)
print(len(sub_ansLIST))

In [ ]:
all_eventsArray = events.copy()
#print(all_eventsArray[0][0])

# C == control group; H == H_CD; L == L_CD
# event_id >> C==9; H==1; L==2

C_LIST = []
H_LIST = []
L_LIST = []

# Change the event id according to the condition
for row in sub_ansLIST:
    #print(row)
    
    index_eventINT = sub_ansLIST.index(row)
    #print(index_eventINT)
    conditionSTR = row[1]
    #print(row[0], row[1])
    
    if conditionSTR == "C":
        #print(index_eventINT, "=", conditionSTR)
        all_eventsArray[index_eventINT][2] = 9
        #C_LIST.append(conditionSTR)

    if conditionSTR == "H":
        #print(index_eventINT, "=", conditionSTR)
        all_eventsArray[index_eventINT][2] = 1
        #H_LIST.append(conditionSTR)
        
    if conditionSTR == "L":       
        #print(index_eventINT, "=", conditionSTR)
        all_eventsArray[index_eventINT][2] = 2
        #L_LIST.append(conditionSTR)
    else:
        pass 

all_eventsArray

In [ ]:
# Creating the new epochs
events = all_eventsArray  #mne.find_events(raw_ICAed, stim_channel='STI 014')  #just checking what stim_channel documenting what.
event_id = {'High_CD':1,'Low_CD':2, 'Control': 9}  

tmin = -0.1         								# pre stimulis interval (in seconds) #
tmax = 0.7          								# post stimulus interval #

baseline = (None, 0)  # tmin ~ 0 , -0.1s-0s   
reject = dict(mag=4e-12)

# Load in the channels layouts
kit_lay = mne.channels.read_layout("KIT-157")

n_LDT_raw.load_data().pick_types(meg=True, stim=False).filter(0, 30, phase= 'zero-double').resample(500)
n_epoch_LDT = mne.Epochs(n_LDT_raw, events, event_id, tmin, tmax,
                    baseline=baseline,
                    reject = reject, preload = True)

n_epoch_LDT

In [ ]:
n_epoch_LDT.save(meg_data_path / Path('S%.3d_ICAed_LDT-epo.fif' %sub_id), overwrite=True)

# EEG ERP (statistics)

### Step 7: Gather all subjects' epochs
### Step 8: Grand average
### Step 9: Draw ERP
### Step 10: Report ERP statistics

In [ ]:
# To see the average evoked of every participants so far

#evoked_C = all_sub_epochsLIST["Control"].average()
evoked_H = all_sub_epochsLIST["High_CD"].average()
evoked_L = all_sub_epochsLIST["Low_CD"].average()

#evoked_C.comment = 'Control'
evoked_H.comment = 'High_CD'
evoked_L.comment = 'Low_CD'

tmp = [evoked_H, evoked_L] # [evoked_H, evoked_L]evoked_C, 
colors = 'blue', 'green'  #C:'red'H:'blue'; L:'green';   # 'blue', 'green'
mne.viz.plot_evoked_topo(tmp, color = colors)

In [ ]:
evoked_L.plot_topomap()

In [ ]:
# Plot the joint
#mne.viz.plot_compare_evokeds(tmp, picks='mag')
H_minus_L = mne.combine_evoked([evoked_H, evoked_L], weights=[1, -1])
H_minus_L.plot_joint()
plt.savefig(meg_data_path / Path('H-minus-L_evoked.png'), overwrite=True)

# Plot the specific times
times = np.linspace(0.05, 0.650, 13)
H_minus_L.plot_topomap(ch_type='mag', times=times, colorbar=True)
plt.savefig(meg_data_path / Path('H-minus-L_evoked_topomap.png'), overwrite=True)

In [ ]:
#evoked_C = n_epoch_LDT["Control"].average()
evoked_H = all_sub_epochsLIST["High_CD"].average()
evoked_L = all_sub_epochsLIST["Low_CD"].average()

times = np.linspace(0.05, 0.45, 9)
#evoked_C.plot_topomap(ch_type='mag', times=times, colorbar=True)
evoked_H.plot_topomap(ch_type='mag', times=times, colorbar=True)
plt.savefig(meg_data_path / Path('HCD-evoked_topomap.png'), overwrite=True)
evoked_L.plot_topomap(ch_type='mag', times=times, colorbar=True)
plt.savefig(meg_data_path / Path('LCD-evoked_topomap.png'), overwrite=True)

In [ ]:
# Check the resluts of some channel
# MEG 043最大/ MEG 048_跟046一樣小/ MEG 045 ＝ MEG 044＿比046大一點/ MEG 046_小/ MEG 047_比046小

mne.viz.plot_compare_evokeds(tmp, picks='MEG 043')
mne.viz.plot_compare_evokeds(tmp, picks='MEG 044')
mne.viz.plot_compare_evokeds(tmp, picks='MEG 045')
mne.viz.plot_compare_evokeds(tmp, picks='MEG 048')

### Grand Average

In [ ]:
# Get the combined epochs to get the data in specific time window for statistics analysis
root_data_path = Path("/Volumes/Neurolang_1/Project_Assistant/2021_Ongoing/2020_LTTC/Experiment_materials/LTTC_MEG")
meg_data_path = root_data_path / "LTTC_MEG_ALL_results"


#print(root_data_path)


"""
n_epoch_LDT = mne.read_epochs(meg_data_path / Path('009-016_LDT-merge-epo.fif'))
n_epoch_LDT.pick('mag')
#n_epoch_LDT = n_epoch_LDT.load_data().get_data()
#print(n_epoch_LDT)
print(len(n_epoch_LDT))

n_epoch_LDT.plot()
"""


tmp_sub_evoked_meanH_LIST = []
all_sub_evoked_meanH_LIST = []

tmp_sub_evoked_meanL_LIST = []
all_sub_evoked_meanL_LIST = []

for idINT in range(9, 17):
    sub_id = idINT
    
    # Load in the epochs of each subs
    n_epoch_LDT = mne.read_epochs(meg_data_path / Path('S%.3d_ICAed_LDT-epo.fif' %sub_id))   
    n_epoch_LDT.pick('mag').crop(0.4, 0.6).pick_channels(["MEG 048"])
    
    # Get the evoked responses from the epochs
    evoked_H = n_epoch_LDT["High_CD"].average()
    evoked_L = n_epoch_LDT["Low_CD"].average()
    
    #print(evoked_H)
    
    # Get the data of certain time range of the evoked, we should pick the certain channel to conduct statistics
    #tmp_evoked_H = evoked_H().copy()
    #tmp_evoked_L = evoked_L().copy()
    tmp_data_H_FLOAT = evoked_H.copy().get_data().mean()
    tmp_data_L_FLOAT = evoked_L.copy().get_data().mean()
    
    print("                         S%.3d_H" %sub_id, tmp_data_H_FLOAT)
    print("                         S%.3d_L" %sub_id, tmp_data_L_FLOAT)
    
    tmp_sub_evoked_meanH_LIST.extend([tmp_data_H_FLOAT])
    tmp_sub_evoked_meanL_LIST.extend([tmp_data_L_FLOAT])
    
    print("                                                   HEREEEEEE!!", len(tmp_sub_evoked_meanH_LIST))
    print("                                                   HEREEEEEE!!", len(tmp_sub_evoked_meanL_LIST))
    
    #all_sub_epochsLIST = mne.concatenate_epochs(tmp_sub_epochsLIST)


In [ ]:
print(tmp_sub_evoked_meanH_LIST)
print(tmp_sub_evoked_meanL_LIST)

In [ ]:
evoked_meanH_Array = np.array(tmp_sub_evoked_meanH_LIST)
print(len(evoked_meanH_Array))
evoked_meanL_Array = np.array(tmp_sub_evoked_meanL_LIST)
print(len(evoked_meanL_Array))

# Conducting pair t-test bwtween HCD & LCD evoked  #default= 雙尾檢定 >>這裡選"less" for單尾檢定 >> 因為是H-L，所以H放前面
pair_ttest_CD = stats.ttest_rel(evoked_meanH_Array, evoked_meanL_Array)#, alternative='less')  
print(pair_ttest_CD)
print(type(pair_ttest_CD))
